## Theory of Dynamic Programming

Dynamic Programming is a method for solving complex problem by breaking down into subproblems and combine solutions of subproblems. Dynamic Programming is a very general solution method for problems which have two properties:
1.  Optimal substructure 
  *  Principle of optimality applies 
  *  Optimal solution can be decomposed into subproblems
2.  Overlapping subprobems
  * Subproblems recur many times
  * Solutions can be cached and reused
  
Markov decicion processes satisfy both properties
  * Bellman equation gives recursive decomposition
  * Value function stores and reuses solutions

In [4]:
! pip install gym

    100% |████████████████████████████████| 1.5MB 6.3MB/s 
    100% |████████████████████████████████| 1.0MB 9.0MB/s 
  Running setup.py bdist_wheel for gym ... - \ | / done
  Stored in directory: /root/.cache/pip/wheels/ea/ec/dd/33bcc8801d345f0b640fced8a0864a7c8474828564bc5ccf70
Successfully built gym


In [6]:
# Use Brute Force to find best solution 
import numpy
import time
import gym

"""
Args:
policy: [S, A] shaped matrix representing the policy.
env: OpenAI gym env.
render: boolean to turn rendering on/off.
"""

#Execution
def execute(env, policy, episodeLength=100, render=False):
  totalReward = 0
  start = env.reset()
  for t in range(episodeLength):
    if render:
      env.render()
    action = policy[start]
    start, reward, done, _ = env.step(action)
    totalReward += reward
    if done:
      break
  return totalReward

#Evaluation
def evaluatePolicy(env, policy, n_episodes=100):
  totalReward = 0.0
  for _ in range(n_episodes):
    totalReward += execute(env, policy)
  return totalReward / n_episodes

#Function for a random policy
def gen_random_policy():
  return numpy.random.choice(4, size=((16)))
 
if __name__ == '__main__':
  env = gym.make('FrozenLake-v0')
  
## Policy search
n_policies = 1000
startTime = time.time()
policy_set = [gen_random_policy() for _ in range(n_policies)]
policy_score = [evaluatePolicy(env, p) for p in policy_set]
endTime = time.time()
print("Best score = %0.2f. Time taken = %4.4f seconds" %(numpy.max(policy_score), endTime - startTime)) 

Best score = 0.35. Time taken = 8.4057 seconds


![Value Iteration](..Img/Value_iteration_qseudocode.PNG)


In [8]:
import numpy
import gym
import time

"""
Args:
policy: [S, A] shaped matrix representing the policy.
env: OpenAI gym env.
env.P represents the transition probabilities of the environment.
env.P[s][a] is a list of transition tuples (prob, next_state, reward, done).
env.nS is a number of states in the environment.
env.nA is a number of actions in the environment.
gamma: Gamma discount factor.
render: boolean to turn rendering on/off.
"""

def execute(env, policy, gamma=1.0, render=False):
  start = env.reset()
  totalReward = 0
  stepIndex = 0
  while True:
    if render:
      env.render() 
    start, reward, done, _ = env.step(int(policy[start]))
    totalReward += (gamma ** stepIndex * reward)
    stepIndex += 1
    if done:
      break
  return totalReward

# Evaluates a policy by running it n times.returns:average total reward
def evaluatePolicy(env, policy, gamma=1.0, n=100):
  scores = [
    execute(env, policy, gamma=gamma, render=False)
    for _ in range(n)]
  return numpy.mean(scores)

# choosing the policy given a value-function
def calculatePolicy(v, gamma=1.0):
  policy = numpy.zeros(env.env.nS)
  for s in range(env.env.nS):
    q_sa = numpy.zeros(env.action_space.n)
    for a in range(env.action_space.n):
      for next_sr in env.env.P[s][a]:
        # next_sr is a tuple of (probability, next state, reward, done)
        p, s_, r, _ = next_sr
        q_sa[a] += (p * (r + gamma * v[s_]))
    policy[s] = numpy.argmax(q_sa)
  return policy

# Value Iteration Agorithm
def valueIteration(env, gamma=1.0):
  value = numpy.zeros(env.env.nS) # initialize value-function
  max_iterations = 10000
  eps = 1e-20
  for i in range(max_iterations):
    prev_v = numpy.copy(value)
    for s in range(env.env.nS): 
      q_sa = [sum([p * (r + prev_v[s_]) for p, s_, r, _ in env.env.P[s][a]]) for a in
range(env.env.nA)]
      value[s] = max(q_sa)
    if (numpy.sum(numpy.fabs(prev_v - value)) <= eps):
      print('Value-iteration converged at # %d.' % (i + 1))
      break
  return value

if __name__ == '__main__':
  gamma = 1.0
  env = gym.make("FrozenLake-v0")
  optimalValue = valueIteration(env, gamma);
  startTime = time.time()
  policy = calculatePolicy(optimalValue, gamma)
  policy_score = evaluatePolicy(env, policy, gamma, n=1000)
  endTime = time.time()
  print("Best score = %0.2f. Time taken = %4.4f seconds" % (numpy.mean(policy_score),
endTime - startTime)) 

Value-iteration converged at # 1373.
Best score = 0.73. Time taken = 0.3815 seconds


## Code Practice 
The exercise of Dynamic Programming is from [this excercise](https://github.com/dennybritz/reinforcement-learning/tree/master/DP/).